# Demo of how to compile/use Cython in a Notebook

This notebook shows an alternative way to compile Cython directly in a Jupyter notebook. This saves you the trouble of needing to write a `setup.py` file, and is handy for prototyping code development before migrating to a module.

These functions are identical to those appearing in the modules in this repo. They are copied-and-pasted here for clarity. Note that the modules have many explanations in the form of in-line comments; those comments are omitted here, so check the modules for further comments. 

In [1]:
import numpy as np

In [2]:
def simple_pairwise_sum_python(arr1, arr2):
    npts1, npts2 = len(arr1), len(arr2)
    result = np.zeros(npts1*npts2)

    for i in range(0, npts1):
        x = arr1[i]

        for j in range(npts2):
            y = arr2[j]

            idx_result = i*npts2 + j
            result[idx_result] = x + y

    return result

To use Cython in a Jupyter notebook, you just need to add the following cell to your notebook, and then add the `%%cython` magic function at the beginning of any cell that compiles cython code. 

In [3]:
%load_ext Cython

In [4]:
%%cython
import numpy as np 
cimport cython 

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
def simple_pairwise_sum_cython(double[:] arr1, double[:] arr2):

    cdef int npts1 = len(arr1)
    cdef int npts2 = len(arr2)

    cdef double[:] result = np.zeros(npts1*npts2, dtype=np.float64)

    cdef int i, j, idx_result
    cdef double x, y

    for i in range(0, npts1):
        x = arr1[i]

        for j in range(npts2):
            y = arr2[j]

            idx_result = i*npts2 + j
            result[idx_result] = x + y

    return np.array(result)

### Create some fake data to demonstrate usage

In [5]:
npts = int(1e3)
arr1 = np.random.random(npts)
arr2 = np.random.random(npts)

### Demonstrate equivalence of cython vs. cython

In [8]:
np.all(simple_pairwise_sum_python(arr1, arr2) == simple_pairwise_sum_cython(arr1, arr2))

True

### Demonstrate performance enhancement

In [11]:
%timeit simple_pairwise_sum_python(arr1, arr2)

1 loop, best of 3: 541 ms per loop


In [12]:
%timeit simple_pairwise_sum_cython(arr1, arr2)

100 loops, best of 3: 9.05 ms per loop
